In [2]:
from math import sin
from random import random
import pandas as pd
import pickle
import json

from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper
from bokeh.palettes import plasma
from bokeh.plotting import figure
from bokeh.transform import transform
from sentence_transformers import SentenceTransformer

import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics

c:\Users\Gebruiker\anaconda3\envs\thesis1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def make_embeds_list(data):
    lst = []
    for row in data:
        model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        embed = model.encode(row)
        lst.append(embed)
        
    return lst

In [4]:
data = pd.read_json('data/train/dataset-000.json')
data_basic = data.drop(['tokens', 'text', 'images'], axis=1)
data_basic

,title,diffbotUri,specs
0,Offering you a complete choice of products whi...,product|4|-1577418857,"{'size': 'All Sizes', 'color': 'Black'}"
1,Acrylic Sheet in Surat,product|4|2080697291,{'specifications': '-'}
2,"Netgear ReadyNAS 312 2-Bay, 2x2TB Desktop Drive",product|4|802308696,"{'form_factor': 'Desktop', 'total_number_of_us..."
3,Stochastic Calculus for Finance II,product|4|2144040105,{'copyright_holder': 'Springer Science+Busines...
4,Stochastic Calculus for Finance II,product|4|2144040105,{'copyright_holder': 'Springer Science+Busines...
...,...,...,...
2168,MOMO MOD. 80 Evo Steering Wheels,product|4|-2023682394,{'color': 'Black'}
2169,Le Creuset Signature 24cm Casserole Volcanic,product|4|-1774049767,"{'colour': 'Orange', 'guarantee': 'Lifetime', ..."
2170,Stainless Steel Spa,product|4|387637073,"{'color_finish': 'Metallic', 'model': 'Custom'..."
2171,Acer America LED KN242HYL UM.QX2AA.001 23.8 in...,product|4|44448056,"{'widescreen': 'Yes', 'brightness': '250 cd/m2'}"


In [5]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

data_basic['keys'] = ''
data_basic['values'] = ''
data_basic['title embeddings'] = ''

i = 0
for row in data_basic['specs']:
    row_keys = list(row.keys())
    row_values = list(row.values())
    data_basic.at[i, 'keys'] = row_keys
    data_basic.at[i, 'values'] = row_values
    i += 1

a = 0
for row in data_basic['title']:
    title_embed = model.encode(row)
    data_basic.at[a, 'title embeddings'] = title_embed
    a += 1

In [5]:
data_basic = data_basic.drop('specs', axis=1)
data_basic

,title,diffbotUri,keys,values,title embeddings
0,Offering you a complete choice of products whi...,product|4|-1577418857,"[size, color]","[All Sizes, Black]","[-0.052266043, 0.34307656, -0.05728973, -0.145..."
1,Acrylic Sheet in Surat,product|4|2080697291,[specifications],[-],"[-0.6556488, 0.460035, -0.0362479, -0.06235854..."
2,"Netgear ReadyNAS 312 2-Bay, 2x2TB Desktop Drive",product|4|802308696,"[form_factor, total_number_of_usb_ports, produ...","[Desktop, 3, 300, Dual-core (2 Core), 2, Seria...","[-0.00080985145, 0.44421533, 0.13045506, -0.05..."
3,Stochastic Calculus for Finance II,product|4|2144040105,[copyright_holder],[Springer Science+Business Media New York],"[0.04476561, -0.3264529, -0.5369208, -0.123263..."
4,Stochastic Calculus for Finance II,product|4|2144040105,[copyright_holder],[Springer Science+Business Media New York],"[0.04476561, -0.3264529, -0.5369208, -0.123263..."
...,...,...,...,...,...
2168,MOMO MOD. 80 Evo Steering Wheels,product|4|-2023682394,[color],[Black],"[0.2821766, 0.041165505, 0.25122377, -0.027214..."
2169,Le Creuset Signature 24cm Casserole Volcanic,product|4|-1774049767,"[colour, guarantee, induction, dishwasher]","[Orange, Lifetime, Yes, Yes]","[-0.33314914, 0.2826587, -0.04688052, -0.34292..."
2170,Stainless Steel Spa,product|4|387637073,"[color_finish, model, essential_properties, ge...","[Metallic, Custom, Green, Traditional, 8, Get ...","[-1.0019083, 0.09988186, -0.07147859, 0.610807..."
2171,Acer America LED KN242HYL UM.QX2AA.001 23.8 in...,product|4|44448056,"[widescreen, brightness]","[Yes, 250 cd/m2]","[0.030759212, 0.29977, 0.23099835, 0.25773454,..."


In [6]:
data_basic['title + key'] = ''

for index, row in data_basic.iterrows():
    keys = ' '.join(row['keys'])
    thing = row['title'] + ' ' + keys
    data_basic.at[index, 'title + key'] = thing

In [100]:
data_basic = data_basic.drop_duplicates(subset=['title'])
data_basic['in_mave'] = [False] * len(data_basic)
data_basic

C:\Users\Nina W\AppData\Local\Temp\ipykernel_8896\3934043273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_basic['in_mave'] = [False] * len(data_basic)


,title,diffbotUri,keys,values,title embeddings,title + key,Cluster_num_title,electronics,in_mave
0,Offering you a complete choice of products whi...,product|4|-1577418857,"[size, color]","[All Sizes, Black]","[-0.052266043, 0.34307656, -0.05728973, -0.145...",Offering you a complete choice of products whi...,-1,False,False
1,Acrylic Sheet in Surat,product|4|2080697291,[specifications],[-],"[-0.6556488, 0.460035, -0.0362479, -0.06235854...",Acrylic Sheet in Surat specifications,-1,False,False
2,"Netgear ReadyNAS 312 2-Bay, 2x2TB Desktop Drive",product|4|802308696,"[form_factor, total_number_of_usb_ports, produ...","[Desktop, 3, 300, Dual-core (2 Core), 2, Seria...","[-0.00080985145, 0.44421533, 0.13045506, -0.05...","Netgear ReadyNAS 312 2-Bay, 2x2TB Desktop Driv...",0,False,False
3,Stochastic Calculus for Finance II,product|4|2144040105,[copyright_holder],[Springer Science+Business Media New York],"[0.04476561, -0.3264529, -0.5369208, -0.123263...",Stochastic Calculus for Finance II copyright_h...,-1,False,False
6,VIVOTEK IB8382-ET 5MP Bullet Network Camera,product|4|-1275148285,"[chip_inch_size, specialist_type, compression_...","[1/3, Network, H.264, MJPEG, Color / Monochrom...","[-0.13799621, -0.07379925, -0.2992541, 0.01109...",VIVOTEK IB8382-ET 5MP Bullet Network Camera ch...,-1,False,False
...,...,...,...,...,...,...,...,...,...
2168,MOMO MOD. 80 Evo Steering Wheels,product|4|-2023682394,[color],[Black],"[0.2821766, 0.041165505, 0.25122377, -0.027214...",MOMO MOD. 80 Evo Steering Wheels color,-1,False,False
2169,Le Creuset Signature 24cm Casserole Volcanic,product|4|-1774049767,"[colour, guarantee, induction, dishwasher]","[Orange, Lifetime, Yes, Yes]","[-0.33314914, 0.2826587, -0.04688052, -0.34292...",Le Creuset Signature 24cm Casserole Volcanic c...,-1,False,False
2170,Stainless Steel Spa,product|4|387637073,"[color_finish, model, essential_properties, ge...","[Metallic, Custom, Green, Traditional, 8, Get ...","[-1.0019083, 0.09988186, -0.07147859, 0.610807...",Stainless Steel Spa color_finish model essenti...,-1,False,False
2171,Acer America LED KN242HYL UM.QX2AA.001 23.8 in...,product|4|44448056,"[widescreen, brightness]","[Yes, 250 cd/m2]","[0.030759212, 0.29977, 0.23099835, 0.25773454,...",Acer America LED KN242HYL UM.QX2AA.001 23.8 in...,0,False,False


In [8]:
## Keys with outliers

# Cluster title embeddings
title_embeds = []

for row in data_basic['title embeddings']:
    title_embeds.append(row)

db = DBSCAN(eps=5, min_samples=5).fit(title_embeds)
title_labels = db.labels_


titles = data_basic['title']
keys = data_basic['keys']
data = list(zip(titles, keys, title_labels))
sorted_titles = pd.DataFrame(data, columns=['titles', 'keys', 'cluster_num'])
sorted_titles = sorted_titles.sort_values(by=['cluster_num'])
sorted_titles.to_csv('clusters/sorted_titles.csv')
sorted_titles


# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(title_labels)) - (1 if -1 in title_labels else 0)
n_noise_ = list(title_labels).count(-1)

print("Estimated number of title clusters: %d" % n_clusters_)
print("Estimated number of title noise points: %d" % n_noise_)


# Separate outliers

data_basic['Cluster_num_title'] = title_labels
sorted_titles = data_basic.sort_values(by=['Cluster_num_title'])
outliers = sorted_titles[sorted_titles.Cluster_num_title == -1]


# Combine outlier titles with key names

outliers['title + key'] = ''

for index, row in outliers.iterrows():
    keys = ' '.join(row['keys'])
    thing = row['title'] + ' ' + keys
    outliers.at[index, 'title + key'] = thing


# Make embeddings of the new titles

title_keys_embeds = []

for row in outliers['title + key']:
    embed = model.encode(row)
    title_keys_embeds.append(embed)


# Cluster again

dbt = DBSCAN(eps=5, min_samples=5).fit(title_keys_embeds)
title_keys_labels = dbt.labels_

outlier_titles = outliers['title']
keys = outliers['keys']
data = list(zip(outlier_titles, keys, title_keys_labels))
sorted_title_keys = pd.DataFrame(data, columns=['outlier_titles', 'keys', 'cluster_num'])
sorted_title_keys = sorted_title_keys.sort_values(by=['cluster_num'])
sorted_title_keys.to_csv('clusters/sorted_title_keys.csv')

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(title_keys_labels)) - (1 if -1 in title_keys_labels else 0)
n_noise_ = list(title_keys_labels).count(-1)

print("Estimated number of title clusters: %d" % n_clusters_)
print("Estimated number of title noise points: %d" % n_noise_)

Estimated number of title clusters: 5
Estimated number of title noise points: 1649


C:\Users\Nina W\AppData\Local\Temp\ipykernel_8896\913996330.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_basic['Cluster_num_title'] = title_labels
C:\Users\Nina W\AppData\Local\Temp\ipykernel_8896\913996330.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outliers['title + key'] = ''


Estimated number of title clusters: 6
Estimated number of title noise points: 1240


In [9]:
sorted_title_keys = sorted_title_keys.sort_values(by=['cluster_num'])
sorted_title_keys.to_csv('clusters/sorted_title_keys.csv')
sorted_title_keys

,outlier_titles,keys,cluster_num
0,Offering you a complete choice of products whi...,"[size, color]",-1
433,Madiera,"[type, price, essential_properties, general_st...",-1
434,Window Regulator,"[payment_condition, delivery_fob_port]",-1
436,The Debt 2010,[discs],-1
439,GXW Wireless 12 MP Scouting Camera,[returnable],-1
...,...,...,...
152,WGT408131 - SATO GT408e Bar code Printer,"[product_type, product_series, serial_port]",5
1633,V7 Black Ink Cartridge For HP DeskJet PSC 1240...,"[printing_color, consumable_type, printing_tec...",5
501,Canon PGI-29M Ink Cartridge - Magenta,"[print_color, http, packaged_quantity, product...",5
1028,Zebra GK42-202210-000 GK420d Desktop Thermal P...,[print_technology],5


## Check if product in MAE also in MAVE

In [98]:
# data read
import gzip
import json    
import os

import pandas as pd


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


In [96]:
df_video_games = getDF('mave/meta_Video_Games.json.gz')
df_video_games = df_video_games.drop(['also_buy', 'tech1', 'tech2', 'brand', 'feature', 'rank', 'similar_item', 'fit', 'also_view', 'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'], axis=1)
df_video_games

,category,description,title,main_cat
0,"[Video Games, PC, Games]",[],Reversi Sensory Challenger,Toys &amp; Games
1,"[Video Games, Xbox 360, Games, </span></span><...",[Brand new sealed!],Medal of Honor: Warfighter - Includes Battlefi...,Video Games
2,"[Video Games, Retro Gaming & Microconsoles, Su...",[],street fighter 2 II turbo super nintendo snes ...,Video Games
3,"[Video Games, Xbox 360, Accessories, Controlle...",[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,Xbox 360 MAS STICK,Video Games
4,"[Video Games, PC, Games, </span></span></span>...","[Phonics Alive! 3, The Speller teaches student...",Phonics Alive! 3: The Speller,Video Games
...,...,...,...,...
84814,"[Video Games, PlayStation Vita, Digital Games ...",[<div>The Force is strong with this one The No...,Lego Star Wars: The Force Awakens - PS Vita [D...,Video Games
84815,"[Video Games, PlayStation 4, Digital Games & D...",[<div>The Season Pass includes three Level Pac...,Lego Star Wars: The Force Awakens Season Pass...,Video Games
84816,"[Video Games, PlayStation 4, Digital Games & D...","[<div>The Technomancer takes you to Mars, wher...",The Technomancer - PS4 [Digital Code],Video Games
84817,"[Video Games, Xbox 360, Accessories, </span></...",[<b>FUNCTIONS:</b><br> 1.Take apart your Xbox ...,"Repair T8 T6 Tools for XBOX One Xbox 360, YTTL...",Video Games


In [157]:

from collections import Counter
#go through all the MAE dataset files
file_num = 0
for file in os.listdir('data'):
    print(file)
    data = pd.read_json(f'data/{file}')
    data = data.drop(['tokens', 'text', 'images'], axis=1)
    data = data.drop_duplicates(subset=['title'])
    data['in_mave'] = [False] * len(data)
    data['category'] = ''
    data['main_cat'] = ''
    data_titles = data['title']

    # go through all csv files with the categories 
    for f in os.listdir('mave/csvs'):
        df = pd.read_csv(f'mave/csvs/{f}')
        # remove duplciates otherwise the category index will be wrong
        df = df.drop_duplicates(subset = 'title')
        df_title = df['title']
        
        # check if a title in the dataset(piece) is in the category
        results = data_titles.isin(df_title)
        results2 = df_title.isin(data_titles)

        # change value in dataset df to show the product is in the mave dataset
        # add the category and main_cat to the item that is in the mave dataset
        categories = []
        main_cats = []
        for index, row in df.iterrows():
            if results2[index] == True:
                category = row['category']
                categories.append(category)
                main_cat = row['main_cat']
                main_cats.append(main_cat)

        num = 0
        for index, row in data.iterrows():
            if results[index] == True:
                data.at[index, 'in_mave'] = True
                data.at[index, 'category'] = categories[num]
                data.at[index, 'main_cat'] = main_cats[num]
                num += 1
    
    print(len(data[data['in_mave'] == True]))
    df_save = data[data['in_mave'] == True] 

    df_save.to_json(f'data_in_mave_{file_num}.json')

    print(file_num)
    file_num += 1
    
    if file_num == 10:
        break
    

dataset-000.json
75
0
dataset-001.json
93
1
dataset-002.json
76
2
dataset-003.json
85
3
dataset-004.json
85
4
dataset-005.json
79
5
dataset-006.json
75
6
dataset-007.json
83
7
dataset-008.json
91
8
dataset-009.json
69
9
